In [ ]:
pip install pipreqs

In [152]:
pip freeze > requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [38]:
# fpath="../../test-cases/test_scenario_template.md"
fpath = "tc1.md"
with open(fpath, "r") as f:
    mtxt = f.read()


In [ ]:
import markdown

ast = markdown.markdown(mtxt)

print(ast)

In [41]:
import re

def parse_markdown(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.readlines()

    summary = {"Headings": [], "Lists": []}

    for line in content:
        # Extract headings
        heading_match = re.match(r"^(#+) (.+)", line)
        if heading_match:
            level = len(heading_match.group(1))  # Number of `#`
            text = heading_match.group(2).strip()
            summary["Headings"].append({"level": level, "text": text})

        # Extract lists
        list_match = re.match(r"^[-*+] (.+)", line)
        if list_match:
            summary["Lists"].append(list_match.group(1).strip())

    return summary

# Example usage
file_path = "tc1.md"  # Replace with your Markdown file path
summary = parse_markdown(file_path)
print("Headings:", summary["Headings"])
print("Lists:", summary["Lists"])


Headings: [{'level': 1, 'text': 'Login success'}, {'level': 2, 'text': 'Title:'}, {'level': 2, 'text': 'Description:'}, {'level': 2, 'text': 'Preconditions:'}, {'level': 2, 'text': 'Steps to Execute:'}, {'level': 2, 'text': 'Expected Outcome:'}, {'level': 2, 'text': 'Impact Area:'}, {'level': 3, 'text': 'frontend'}, {'level': 3, 'text': 'backend api'}, {'level': 2, 'text': 'Type of Test:'}, {'level': 2, 'text': 'Tags:'}, {'level': 3, 'text': 'features'}, {'level': 3, 'text': 'execution'}, {'level': 3, 'text': 'impact'}, {'level': 3, 'text': 'type'}, {'level': 2, 'text': 'Affected Versions:'}, {'level': 2, 'text': 'Attachments:'}, {'level': 2, 'text': 'Average Performance Time:'}, {'level': 3, 'text': 'Test:'}, {'level': 3, 'text': 'Browser:'}, {'level': 4, 'text': 'network calls:'}, {'level': 3, 'text': 'Lambda:'}, {'level': 4, 'text': 'user-login'}, {'level': 2, 'text': 'Notes/Comments:'}]
Lists: ['The loading indicator is displayed for few seconds (~5).', 'user is navigated to `secur

In [82]:
import re

with open("tc1.md", "r", encoding="utf-8") as file:
    tc1_markdown_lines = file.readlines()


In [79]:
def parse_metadata(lines):
    metadata = None

    for line in lines:
        if line.strip() == "---":
            if metadata == None:
                metadata = dict()
            else:
                break
        else:
            metadata_match = re.match(r"(.+):(.+)", line)
            if metadata_match:
                key = metadata_match.group(1).strip()
                text = metadata_match.group(2).strip()
                if key in metadata:
                    raise ValueError("duplicate metadata key")
                metadata[key] = text
    return metadata

print(parse_metadata(tc1_markdown_lines))
 

({'id': 'login-logout-tc1'}, 3)


In [ ]:
import json
from typing import List
from markdown_to_json import dictify


def get_formatted_key(k:str):
    kr = k.replace("`", "").strip()
    if kr.endswith(":"):
        return kr[:-1]
    return kr

def flatten_list_dict(list_values: List, depth_level:str|None="max"):
    if isinstance(list_values, List):
        list_dict = dict()
        for lv in list_values:
            if not isinstance(lv, dict):
                print("list value {lv} is not dictionary. cannot flatten list of dict to dict of key-values")
                list_dict = None
                break
            if len(lv.keys()) != 1:
                print("dictionary value in list {lv} doesnot match required only 1 key. cannot flatten list of dict to dict of key-values")
                list_dict = None
                break
            k,v = lv.popitem()
            fv = None
            if depth_level == "max":
                fv = flatten_list_dict(v, depth_level)

            list_dict[k] = fv if fv != None else v
        return list_dict
    return list_values

def convert_list_to_dict(key:str, list_values:List|str|dict):
    list_dict = dict()

    if isinstance(list_values, dict):
        for k,v in list_values.items():
            kk = get_formatted_key(k)
            list_dict[kk] = convert_list_to_dict(kk, v)
        return list_dict

    if isinstance(list_values,str):
        list_values = [list_values]

    if not isinstance(list_values, List):
        raise TypeError(f"{key} are not in list format")

    for lv in list_values:
        if not isinstance(lv, str):
            raise TypeError(f"value [{lv}] in list is not string")

        lv_array = lv.split("=")
        if len(lv_array) != 2:
            raise AssertionError(f"{key} key and value are in incorrect format. it should be tagkey=value1,value2")

        dict_k = get_formatted_key(lv_array[0])
        if dict_k in list_dict:
            raise AssertionError(f"the key[{dict_k}] for {key} is duplicate")
        
        list_dict[dict_k] = get_formatted_key(lv_array[1]).split(",")
    return list_dict

filename = "tc1.md"

def convert_tc(lines: List[str]):
    metadata,ln = parse_metadata(lines)
    
    if "id" not in metadata:
        raise AssertionError("id is missing in metadata of " + filename)
    converted_tc_dict = dict()
    converted_tc_dict[metadata["id"]] = dict()
    converted_tc_dict[metadata["id"]]["metadata"] = metadata
    
    tc_dict = dictify("\n".join(lines))
    if not isinstance(tc_dict, dict):
        raise TypeError("test case details are not converted into dictionary")

    if len(tc_dict.keys()) != 1:
        raise AssertionError(f"test case file format is incorrect. only 1 level headings supported. but found [{ ','.join(tc_dict.keys() ) }]")
    
    k,v = tc_dict.popitem()
    print(f"key={k} and value={v}")
    details_dict = converted_tc_dict[metadata["id"]]["details"] = dict()
    for k1,v1 in v.items():
        # print(k1,v1)
        details_k = get_formatted_key(k1)
        details_v = v1
        if details_k == "Tags":
            details_v = convert_list_to_dict(details_k, v1)
        elif details_k == "Average Performance Time":
            if not isinstance(v1, dict):
                raise TypeError(f"{details_k} is not dictionary")

            details_v = convert_list_to_dict(details_k, v1)
        
        details_dict[details_k]= details_v

    return converted_tc_dict

print(tc1_markdown_lines)
print("converted test case details =", json.dumps(convert_tc(tc1_markdown_lines), indent=4))


In [34]:
# fpath="../../test-cases/test_scenario_template.md"
fpath = "converted-tc1.md"
with open(fpath, "r") as f:
    cmtxt = f.read()


In [ ]:
from markdown_to_json import dictify

dict_data = dictify(cmtxt)

print(dict_data)



OrderedDict([('Login success', OrderedDict([('Title:', 'User login successful after logout'), ('Description:', 'logged in user tries to logout successful. upon logging out, user tries to login again. when successful user can access all secured features e.g. add purchase, view settings, etc.'), ('Preconditions:', 'User is on login page.'), ('Steps to Execute:', ['user fills out details as following,', ['emailId: `sardar.vallabhbhai.patel@example.com`', 'password: `$Ardar123`'], 'user clicks on login button', 'login is successful, and logout link is displayed.', 'user clicks on logout link', 'user logs out successfully.', 'user attempts login with same credential immediately']), ('Expected Outcome:', ['The loading indicator is displayed for few seconds (~5).', 'user is navigated to `secured homepage`.', 'secured links like, expenses, payment accounts, settings, logout, etc. are showing and navigational.']), ('Impact Area:', OrderedDict([('frontend', ['login page', 'homepage', 'logout pag

In [37]:
from markdown_to_json import jsonify

json_data = jsonify(cmtxt)

print(json_data)

f = open("converted-tc1.json", "w")
f.write(json_data)
f.close()

{"login-logout-tc1": {"Title:": ["Login success", "User login successful after logout"], "Description:": "logged in user tries to logout successful. upon logging out, user tries to login again. when successful user can access all secured features e.g. add purchase, view settings, etc.", "Preconditions:": "User is on login page.", "Steps to Execute:": ["user fills out details as following,", ["emailId: `sardar.vallabhbhai.patel@example.com`", "password: `$Ardar123`"], "user clicks on login button", "login is successful, and logout link is displayed.", "user clicks on logout link", "user logs out successfully.", "user attempts login with same credential immediately"], "Expected Outcome:": ["The loading indicator is displayed for few seconds (~5).", "user is navigated to `secured homepage`.", "secured links like, expenses, payment accounts, settings, logout, etc. are showing and navigational."], "Impact Area:": {"frontend": ["login page", "homepage", "logout page"], "backend api": ["login

In [ ]:
from pathlib import Path

p=Path("../../../test-cases/")

print("absolute=",p.absolute())
print("cwd=",p.cwd())

for f in p.rglob("*.md"):
    print(f.relative_to(p))

In [147]:
import traceback


def testtry(fail=False):
    try:
        if fail == True:
            raise AssertionError("it is failed")
        print("there is no error")
    except Exception as e:
        print("something has gone wrong.", e)
        traceback.print_exception(e)

testtry()

testtry(True)

there is no error
something has gone wrong. it is failed


Traceback (most recent call last):
  File "C:\Users\neel4\AppData\Local\Temp\ipykernel_33776\4165521157.py", line 7, in testtry
    raise AssertionError("it is failed")
AssertionError: it is failed
